In [88]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf

from sklearn import decomposition, preprocessing
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# Enoncé

Vous avez participé à une compétition sur Kaggle sur le jeu de données de Titanic (celle-ci existe, les curieux peuvent la retrouver sur Kaggle !). Vous avez pour cela à votre disposition [une liste de 891 passagers](https://gist.githubusercontent.com/michhar/2dfd2de0d4f8727f873422c5d959fff5/raw/fa71405126017e6a37bea592440b4bee94bf7b9e/titanic.csv), contenant les caractéristiques suivantes :

- PassengerID : Identifiant du passager
- Survived : Indicateur de survi d'un passager (1 si le passager a survecu, 0 s’il est décédé)
- Pclass: Classe du passager (1 = 1ère classe, 2 = 2ème classe, 3 = 3ème classe)
- Name : Nom et titre du passager
- Sex : Sexe du passager
- Age : Age du passager (Décimal si inférieur à 1, estimé si de la forme xx.5)
- SibSp : Nombre d’époux, de frères et de soeurs présents à bord
- Parch : Nombre de parents ou d’enfants présents à bord 
- Ticket : Numéro du ticket 
- Fare : Tarif des tickets (Le prix est indiqué en £ et pour un seul achat (peut correspondre à plusieurs tickets)
- Cabin : Numéro de Cabine
- Embarked : Port d’embarcation (C = Cherbourg, Q = Queenstown, S = Southampton)
	 	

# Exercice

La compétition a été l’occasion de revenir sur ce jeu de données très célèbre, et plusieurs tâches étaient attendues, :
- identifier les facteurs favorisants la survie d'un passager par rapport à un autre, en dressant une typologie des survivants ;
- créer un algorithme qui pourrait prédire la survie d'un individu à partir de ces caractéristiques.

Vous avez été ajouté à une équipe et le travail et lancé depuis quelques semaines : à cette étape,  la mission est en réalité finie (c.f. le présent notebook). Vos co-équipiers ont travaillé dur : il faut dans un premier temps vous approprier leur travail.


## 1. Traitements

In [105]:
df = pd.read_csv('titanic_dataset.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Analyse et traitement des valeurs manquantes :

In [106]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

De nombreuses valeurs manquantes parsèment le jeu de données. 

Le nombre sur **Cabin** peut potentiellement facilement s'expliquer assez facilement : cela correspond pour une grande partie à des individus qui n'avaient simplement pas de cabine. En effet, seuls les passagers de 1ère classe et certains de 2ème disposaient d’un numéro de cabine pour le voyage. Ces cabines étaient répartis entre les ponts A et E pour les 1ère classe, et de D à G pour les autres. La cabine 'T' est une cabine de classe supérieure située sur le pont A.

Si on observe rapidement la variable Cabin, on peut noter que le première caractère correspond au pont en question. On extrait donc ce dernier et on analyse la répartition des classes au sein des valeurs manquantes sur notre pont :

In [107]:
#df['Cabin'].str[0]

In [108]:
df['Deck'] = df['Cabin'].str[0]

In [109]:
#df['Deck']

In [110]:

df.loc[df['Deck'] == 'T','Deck'] = 'A'

In [111]:
df.loc[df['Deck'].isnull(),'Pclass'].value_counts()

3    479
2    168
1     40
Name: Pclass, dtype: int64

Comme attendu, on retrouve une majorité de 2ème et 3ème classe : pour ces derniers, on peut leur attribuer la valeur N qui correspondra à l'absence d'attribution

In [112]:
df.loc[(df['Deck'].isnull()) & (df['Pclass'].isin([2, 3])),'Deck'] = 'N'

Pour les 1ères classes restantes, on utilisera la moyenne payée par leurs homologues en fonction de leur pont attribué et de leur pont d'embarcation :

In [114]:
prixDeck = df.loc[df['Pclass']==1,['Deck', 'Embarked','Fare']].groupby(['Deck','Embarked']).mean().reset_index()
prixDeck

,Deck,Embarked,Fare
0,A,C,38.357743
1,A,S,40.150456
2,B,C,145.964018
3,B,S,85.372283
4,C,C,98.582533
5,C,Q,90.000000
6,C,S,101.630442
7,D,C,85.586000
8,D,S,49.719906
9,E,C,92.905840


A partir du tableau ci dessus, on attribue à chaque personne le pont ayant le tarif le plus proche de celui qu'elle a payé, en fonction de l'endroit de son port d'embarcation :

In [98]:
mask = df['Deck'].isnull()

In [99]:
mask

0      False
1      False
2      False
3      False
4      False
       ...  
886    False
887    False
888    False
889    False
890    False
Name: Deck, Length: 891, dtype: bool

In [115]:
mask = df['Deck'].isnull()

def get_closest(Fare, Embarked, prixDeck):
    prixDeck = prixDeck.loc[prixDeck['Embarked'] == Embarked,:]
    return prixDeck.loc[prixDeck['Fare'].sub(Fare).abs().idxmin(), 'Deck']

df.loc[mask,'Deck'] = df.loc[mask,:].apply(lambda x: get_closest(x['Fare'], x['Embarked'], prixDeck), axis=1)

df.isnull().sum() # plus de valeurs manquantes sur notre pont, on pourra supprimer la variable Cabine après coup

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
Deck             0
dtype: int64

In [116]:
df.loc[(df['Pclass']==1),:][['Deck', 'Embarked','Fare','Pclass','Cabin']].sort_values(by='Fare')

,Deck,Embarked,Fare,Pclass,Cabin
633,A,S,0.0000,1,NaN
822,A,S,0.0000,1,NaN
815,B,S,0.0000,1,B102
806,A,S,0.0000,1,A36
263,B,S,0.0000,1,B94
...,...,...,...,...,...
88,C,S,263.0000,1,C23 C25 C27
341,C,S,263.0000,1,C23 C25 C27
679,B,C,512.3292,1,B51 B53 B55
258,B,C,512.3292,1,NaN


En faisant une analyse rapide des personnes dont il manque le port d'embarcation, et en utilisant le tableau **prixDeck**, étant toutes deux de classes 1 et étant sur le pont B, on peut décemment supposer qu'elles ont embarqué à Southampton (S)

In [58]:
df.loc[df['Embarked'].isnull(),:]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck
61,62,1,1,"Icard, Miss. Amelie",female,38.0,0,0,113572,80.0,B28,NaN,B
829,830,1,1,"Stone, Mrs. George Nelson (Martha Evelyn)",female,62.0,0,0,113572,80.0,B28,NaN,B


In [59]:
df.loc[df['Embarked'].isnull(),'Embarked'] = 'S'

In [60]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Deck             0
dtype: int64

Age étant la dernière variable comportant des valeurs manquantes, on peut envisager une régression linéaire pour remplacer ses valeurs manquantes, en utilisant l'ensemble des autres informations. Il faut cependant d'abord retirer les variables que nous n'utiliserons pas ou plus (ID, Name, Ticket et Cabin)

In [61]:
df = df.drop(columns=['PassengerId','Name','Ticket','Cabin'])

In [62]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Deck'],
      dtype='object')

On complète ensuite la variable Age :

In [63]:
#df.info()

In [64]:
#round(9.9,0)

In [65]:
#df.loc[~mask,:].drop(columns=['Age'])

In [71]:
mask = df['Age'].notnull()

lr = smf.ols(formula='Age ~ Survived + Pclass + Sex + SibSp + Parch + Fare + Embarked + Deck',
             data=df.loc[mask,:]).fit()

df.loc[~mask, 'Age'] = round(lr.predict(df.loc[~mask,:].drop(columns=['Age'])),0)
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
Deck        0
dtype: int64

On peut créer une variable NbFam qui correspondra à la somme de SibSp et Parch et changer notre variable représentant la classe, en catégorie

In [72]:
df['NbFam'] = df['SibSp'] + df['Parch']
df['Pclass'] = df['Pclass'].astype('category')
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Deck,NbFam
0,0,3,male,22.0,1,0,7.2500,S,N,1
1,1,1,female,38.0,1,0,71.2833,C,C,1
2,1,3,female,26.0,0,0,7.9250,S,N,0
3,1,1,female,35.0,1,0,53.1000,S,C,1
4,0,3,male,35.0,0,0,8.0500,S,N,0


Nous avons fini avec le traitement de notre base, on peut à présenter passer à la partie analyse et modélisation

## 2. Analyse 